In [1]:
import tensorflow as tf
import numpy as np

In [5]:
int2binary ={}#int2binary[3]=[0,0,0,0,0,0,1,1]
binary_dim = 8

largest_number = pow(2,binary_dim)

binary = np.unpackbits(
           np.array([range(largest_number)],dtype=np.uint8).T,axis = 1)
#建立字典
for i in range(largest_number):
    int2binary[i] = binary[i]
    
def binary_generation(numbers,reverse = False):
    binary_x = np.array([int2binary[num] for num in numbers],dtype=np.uint8)
    
    if reverse:
        binary_x = np.fliplr(binary_x)
    return binary_x

def batch_generation(batch_size,largest_number):
#     随机生成batch_size 个数
        n1 = np.random.randint(0,largest_number//2,batch_size)
        n2 = np.random.randint(0,largest_number//2,batch_size)
        add = n1 + n2
        
        binary_n1 = binary_generation(n1,True)
        binary_n2 = binary_generation(n2,True)
        binary_n3 = binary_generation(add,True)
        batch_y = binary_generation(add,True)
        
        batch_x = np.dstack((binary_n1,binary_n2))
        return batch_x,batch_y,n1,n2,add

def binary2int(binary_array):
    out = 0
    for index,x in enumerate(reversed(binary_array)):
        out += x*pow(2,index)
    return out

In [6]:
batch_size  = 64
lstm_size = 20
lstm_layers = 2

In [7]:
x = tf.placeholder(tf.float32,[None,binary_dim,2],name='input_x')

y_ = tf.placeholder(tf.float32,[None,binary_dim],name='input_y')

keep_prob = tf.placeholder(tf.float32,name='keep_prob')

In [10]:
#建立模型
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

drop = tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)

def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(lstm_size)
cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])

initial_state = cell.zero_state(batch_size,tf.float32)

outputs, final_state = tf.nn.dynamic_rnn(cell,x,initial_state=initial_state)

# build output layer
weights  = tf.Variable(tf.truncated_normal([lstm_size,1],stddev=0.01))
bias = tf.zeros([1])

outputs = tf.reshape(outputs,[-1,lstm_size])

logits = tf.sigmoid(tf.matmul(outputs,weights))

predictions = tf.reshape(logits,[-1,binary_dim])


In [11]:
cost = tf.losses.mean_squared_error(y_,predictions)
optimizer = tf.train.AdamOptimizer().minimize(cost)

In [13]:
#train
steps = 2000
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    iteration = 1
    for i in range(steps):
        input_x,input_y,_,_,_ = batch_generation(batch_size,largest_number)
        _,loss = sess.run([optimizer,cost],feed_dict = {x:input_x,y_:input_y,keep_prob:0.5})
        
        if iteration % 1000 == 0:
            print("iter:{},Loss:{}".format(iteration,loss))
        iteration +=1
#     训练结束 ，进行测试
    val_x,val_y,n1,n2,add = batch_generation(batch_size,largest_number)
    result = sess.run(predictions,feed_dict={x:val_x,y_:val_y,keep_prob:1.0})
    
    result = np.fliplr(np.round(result))
    result = result.astype(np.int32)
    
    for b_x,b_p,a,b,add in zip(np.fliplr(val_x),result,n1,n2,add):
        print('{}:{}'.format(b_x[:,0],a))
        print('{}:{}'.format(b_x[:,1],b))
        print('{}:{}\n'.format(b_p,binary2int(b_p)))

iter:1000,Loss:0.03537212312221527
iter:2000,Loss:0.0010282548610121012
[0 1 1 1 0 0 0 0]:112
[0 1 1 1 0 1 0 0]:116
[1 1 1 0 0 1 0 0]:228

[0 0 1 0 0 0 0 1]:33
[0 0 1 0 1 0 1 0]:42
[0 1 0 0 1 0 1 1]:75

[0 0 0 1 0 1 1 0]:22
[0 0 0 0 1 1 1 1]:15
[0 0 1 0 0 1 0 1]:37

[0 1 1 1 1 1 1 1]:127
[0 0 0 1 0 1 0 1]:21
[1 0 0 1 0 1 0 0]:148

[0 1 0 1 0 1 0 0]:84
[0 1 1 1 1 0 1 1]:123
[1 1 0 0 1 1 1 1]:207

[0 0 1 1 0 0 0 0]:48
[0 1 0 1 0 0 0 1]:81
[1 0 0 0 0 0 0 1]:129

[0 1 1 0 0 1 0 1]:101
[0 1 0 0 1 0 1 1]:75
[1 0 1 1 0 0 0 0]:176

[0 1 1 0 0 1 1 1]:103
[0 0 0 1 0 1 1 0]:22
[0 1 1 1 1 1 0 1]:125

[0 1 1 0 1 0 1 0]:106
[0 0 1 1 0 1 1 0]:54
[1 0 1 0 0 0 0 0]:160

[0 0 1 0 1 1 1 0]:46
[0 1 0 1 1 1 1 0]:94
[1 0 0 0 1 1 0 0]:140

[0 1 1 0 1 1 1 1]:111
[0 1 1 1 1 1 0 1]:125
[1 1 1 0 1 1 0 0]:236

[0 1 1 0 0 0 1 0]:98
[0 1 1 1 0 1 0 0]:116
[1 1 0 1 0 1 1 0]:214

[0 0 1 1 0 1 1 1]:55
[0 1 0 1 0 1 0 0]:84
[1 0 0 0 1 0 1 1]:139

[0 1 0 0 1 0 1 1]:75
[0 1 1 1 1 1 0 0]:124
[1 1 0 0 0 1 1 1]:199

[0 0 1 1 